In [1]:
import pandas as pd
import ast
import numpy as np
from IPython.display import display, clear_output, HTML
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt

# Similarity Scheme
Each product's ingredients will be given a weight based on its position in the list. To generate the similarity score between two different products, each matched ingredient's weights will be multiplied together, and then all of those will be summed. Finally, this sum is divided by the maximum possible score (which is the weights from the shorter list squared and then summed).

To get the weights, we will use the geometric distribution.

![Geometric Distribution (from Wikipedia)](images/geometric_distribution.png)

**Geometric distribution:**
The probability that the first occurrence of success requires $i$ independent trials each with success probability $p$.
$$
p(1 - p) ^ {i - 1}
$$
A smaller p will produce more evenly distributed weights, whereas a larger p will put more importance on earlier ingredients.

In [2]:
# uses a geometric distribution so each weight
# decreases geometrically according to its position.
def generate_weights(n: int, p: float = 0.2) -> list:
    '''
    n: length of desired weight list
    p: parameter for geometric distribution (between 0 and 1)

    Returns a list of weights that sum to 1 based on the
    geometric distribution.
    '''
    weights = []
    total_weight = 0
    for i in range(1, n + 1):
        weight = p * ((1 - p) ** (i - 1)) # geometric pdf
        weights.append(weight)
        total_weight += weight
    normalized_weights = [weight / total_weight for weight in weights]
    return normalized_weights

generate_weights(5, 0.2)

[0.29747739171822934,
 0.23798191337458352,
 0.19038553069966682,
 0.15230842455973348,
 0.12184673964778678]

## Weighted Distance Matrix
The goal of this similarity scheme is to produce a distance matrix that judges the "distance" of the products based on the geometric weights assigned to each ingredient. We will use every unique ingredient included in the dataset so that each product has a weight assigned to each possible ingredient (if it doesn't have a particular ingredient, the weight is 0).

First, we need to import the data (formatted in `0_data_preprocess.ipynb`) and add the weights column.

In [3]:
# Dataframe exported/imported as a pickle to preserve 
# the columns with a list format (csv gets messy)
df = pd.read_pickle('data/skincare_products_listed.pickle')
df.head(3)

,Label,Brand,Name,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive,Cosing Ref No,INCI Name,Function
0,Moisturizer,LA MER,Crème de la Mer,175,4.1,"[Algae (Seaweed) Extract, Mineral Oil, Petro...",1,1,1,1,1,"[54290.0, 95058.0, 79504.0, 34040.0, 34654.0, ...","[ALGAE EXTRACT, HYDROGENATED MINERAL OIL, PETR...","[FRAGRANCE, HUMECTANT, ORAL CARE, SKIN CONDITI..."
1,Moisturizer,SK-II,Facial Treatment Essence,179,4.1,"[Galactomyces Ferment Filtrate (Pitera), Buty...",1,1,1,1,1,"[84397, 74756, 58983, 92472, 37735, 35342, 38173]","[GALACTOMYCES FERMENT FILTRATE, BUTYLENE GLYCO...","[HUMECTANT, FRAGRANCE, HUMECTANT, SKIN CONDITI..."
2,Moisturizer,DRUNK ELEPHANT,Protini™ Polypeptide Cream,68,4.4,"[Water, Dicaprylyl Carbonate, Glycerin, Cet...",1,1,1,1,0,"[92472, 55832, 34040, 75132, 55337, 38182, 583...","[WATER, DICAPRYLYL CARBONATE, GLYCERIN, CETEAR...","[ANTIPLAQUE, SKIN CONDITIONING, SOLVENT, SKIN ..."


In [4]:
# Adds weights column to df
df['Weights'] = \
    df['INCI Name'].apply(lambda x: generate_weights(len(x), p=0.2))

df.head(3)

,Label,Brand,Name,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive,Cosing Ref No,INCI Name,Function,Weights
0,Moisturizer,LA MER,Crème de la Mer,175,4.1,"[Algae (Seaweed) Extract, Mineral Oil, Petro...",1,1,1,1,1,"[54290.0, 95058.0, 79504.0, 34040.0, 34654.0, ...","[ALGAE EXTRACT, HYDROGENATED MINERAL OIL, PETR...","[FRAGRANCE, HUMECTANT, ORAL CARE, SKIN CONDITI...","[0.20002126990973731, 0.16001701592778989, 0.1..."
1,Moisturizer,SK-II,Facial Treatment Essence,179,4.1,"[Galactomyces Ferment Filtrate (Pitera), Buty...",1,1,1,1,1,"[84397, 74756, 58983, 92472, 37735, 35342, 38173]","[GALACTOMYCES FERMENT FILTRATE, BUTYLENE GLYCO...","[HUMECTANT, FRAGRANCE, HUMECTANT, SKIN CONDITI...","[0.25307332242756025, 0.2024586579420482, 0.16..."
2,Moisturizer,DRUNK ELEPHANT,Protini™ Polypeptide Cream,68,4.4,"[Water, Dicaprylyl Carbonate, Glycerin, Cet...",1,1,1,1,0,"[92472, 55832, 34040, 75132, 55337, 38182, 583...","[WATER, DICAPRYLYL CARBONATE, GLYCERIN, CETEAR...","[ANTIPLAQUE, SKIN CONDITIONING, SOLVENT, SKIN ...","[0.20000038312461918, 0.16000030649969535, 0.1..."


Now, we can define a function that will create a weight matrix out of the dataframe as it is currently formatted. 

In [5]:
def create_weight_matrix(df):
    '''
    Creates a (n,m) array, with n rows of
    products each containing weights for each of
    the m unique ingredients.
    '''
    names = df['Name']

    ingredients = df.explode("INCI Name")['INCI Name'].unique()
    ingredients.sort()

    weight_matrix = \
        np.zeros((len(names), len(ingredients)))
    
    curr_row = 0

    for i in df.index:
        curr_ingredient_vector = np.zeros(len(ingredients))

        indices = np.searchsorted(
            ingredients, df.loc[i, 'INCI Name'])
        
        curr_ingredient_vector[indices] = \
            df.loc[i, 'Weights']
        
        weight_matrix[curr_row, :] = curr_ingredient_vector
        curr_row += 1

    return weight_matrix 


In [6]:
# 1248 products, 3248 unique ingredients
weight_matrix = create_weight_matrix(df)
weight_matrix.shape

(1248, 3248)

In [7]:
# Saved in data/weighted_distance_matrix.npy
# Otherwise, takes ~1min to compute.
# Uncomment to recompute.

#weighted_distance_matrix = distance_matrix(weight_matrix, weight_matrix)
#np.save('data/weighted_distance_matrix.npy', weighted_distance_matrix)

In [12]:
weighted_distance_matrix = np.load("data/weighted_distance_matrix.npy")
weighted_distance_matrix[0:3, 0:3]

array([[0.        , 0.53000315, 0.442218  ],
       [0.53000315, 0.        , 0.47889144],
       [0.442218  , 0.47889144, 0.        ]])

As expected, diagonals will all be 0 (the closest distance for each product is itself). Diagonals will also be duplicates.

In [13]:
# Creates a list of indices that would sort row 1 (aka, product 2) in
# ascending order, i.e. which products are "closest"
sorted_indices = sorted(range(len(weighted_distance_matrix[1])),
                        key = lambda i: weighted_distance_matrix[1][i])

sorted_indices[0:5] 

[1, 33, 103, 734, 128]

As expected, the most similar product is itself (ID 1).

In [14]:
# Shows the scores associated with the top 5 closest
# products for product no. 2 (including itself)
weighted_distance_matrix[1][sorted_indices[0:4]]

array([0.        , 0.        , 0.07237208, 0.27207928])

The smallest score will always be 0 for its distance from itself. Here, though, we see another 0. We can look at which products are associated with these scores to see why that is.

In [44]:
# Gives the product names for the top 5 closest
# products.
most_similar_products = df.iloc[sorted_indices[1:6],:]['Name'].values
most_similar_products

array(['Facial Treatment Essence Mini',
       'Facial Treatment Essence Karan Singh Limited Edition',
       'Brightening Derm Revival Mask', 'GenOptics Spot Essence Serum',
       'GenOptics Aura Essence Serum'], dtype=object)

As we can see from the results, the product that also had a score of 0 was the mini version of the same product (that tracks).
Now we want to add the top 5 similar products to the dataframe (besides itself). Now, we can put these steps into a function that we can apply to the dataframe.

In [42]:
def get_5_most_similar_products(weighted_distance_matrix: np.ndarray, index: int) -> np.ndarray:
    '''
    weighted_distance_matrix: the distance matrix including the
    similarity "distances" of products
    index: the positional index of the product
    Gets the 5 most similar products (aside from itself) of the 
    product at the given position using the weighted distance matrix. 
    Returns a list.
    '''
    sorted_indices = sorted(range(len(weighted_distance_matrix[index])),
                        key = lambda i: weighted_distance_matrix[index][i])
    
    most_similar_products = df.iloc[sorted_indices[1:6],:]['Name'].values

    return most_similar_products

In [84]:
df2 = df.reset_index()
df2['Weighted Similarity Products'] = df.reset_index().reset_index()['level_0'].apply(lambda x: get_5_most_similar_products(weighted_distance_matrix, x))
df2

,index,Label,Brand,Name,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive,Cosing Ref No,INCI Name,Function,Weights,Weighted Similarity Products
0,0,Moisturizer,LA MER,Crème de la Mer,175,4.1,"[Algae (Seaweed) Extract, Mineral Oil, Petro...",1,1,1,1,1,"[54290.0, 95058.0, 79504.0, 34040.0, 34654.0, ...","[ALGAE EXTRACT, HYDROGENATED MINERAL OIL, PETR...","[FRAGRANCE, HUMECTANT, ORAL CARE, SKIN CONDITI...","[0.20002126990973731, 0.16001701592778989, 0.1...",[Little Miss Miracle Limited-Edition Crème de ...
1,1,Moisturizer,SK-II,Facial Treatment Essence,179,4.1,"[Galactomyces Ferment Filtrate (Pitera), Buty...",1,1,1,1,1,"[84397, 74756, 58983, 92472, 37735, 35342, 38173]","[GALACTOMYCES FERMENT FILTRATE, BUTYLENE GLYCO...","[HUMECTANT, FRAGRANCE, HUMECTANT, SKIN CONDITI...","[0.25307332242756025, 0.2024586579420482, 0.16...","[Facial Treatment Essence Mini, Facial Treatme..."
2,2,Moisturizer,DRUNK ELEPHANT,Protini™ Polypeptide Cream,68,4.4,"[Water, Dicaprylyl Carbonate, Glycerin, Cet...",1,1,1,1,0,"[92472, 55832, 34040, 75132, 55337, 38182, 583...","[WATER, DICAPRYLYL CARBONATE, GLYCERIN, CETEAR...","[ANTIPLAQUE, SKIN CONDITIONING, SOLVENT, SKIN ...","[0.20000038312461918, 0.16000030649969535, 0.1...","[C-Tango™ Multivitamin Eye Cream, After-Sun Mi..."
3,3,Moisturizer,LA MER,The Moisturizing Soft Cream,175,3.8,"[Algae (Seaweed) Extract, Cyclopentasiloxane,...",1,1,1,1,1,"[54290.0, 75413.0, 79504.0, 34067.0, 79701.0, ...","[ALGAE EXTRACT, CYCLOPENTASILOXANE, PETROLATUM...","[FRAGRANCE, HUMECTANT, ORAL CARE, SKIN CONDITI...","[0.20000000862718334, 0.16000000690174668, 0.1...","[The Moisturizing Soft Lotion, The Concentrate..."
4,4,Moisturizer,IT COSMETICS,Your Skin But Better™ CC+™ Cream with SPF 50+,38,4.1,"[Water, Snail Secretion Filtrate, Phenyl Tri...",1,1,1,1,1,"[92472.0, 58704.0, 79701.0, 33401.0, 74756.0, ...","[WATER, SNAIL SECRETION FILTRATE, PHENYL TRIME...","[ANTIPLAQUE, SKIN CONDITIONING, SOLVENT, SKIN ...","[0.20000000353369418, 0.16000000282695537, 0.1...",[Your Skin But Better™ CC+Illumination™ Cream ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,1466,Sun protect,KATE SOMERVILLE,Daily Deflector™ Moisturizer Broad Spectrum SP...,48,3.9,"[Water, Butylene Glycol, PEG-8, Glycerin, ...",0,0,0,0,0,"[92472.0, 74756.0, 77303.0, 34040.0, 85994.0, ...","[WATER, BUTYLENE GLYCOL, PEG-8, GLYCERIN, UNDE...","[ANTIPLAQUE, SKIN CONDITIONING, SOLVENT, FRAGR...","[0.2000265880940796, 0.1600212704752637, 0.128...",[Neuropeptide Firming & Illuminating Under-Eye...
1244,1467,Sun protect,KORRES,Yoghurt Nourishing Fluid Veil Face Sunscreen B...,35,3.9,"[Water, Alcohol Denat., Potassium Cetyl Phos...",1,1,1,1,1,"[92472.0, 74174.0, 79512.0, 34040.0, 74860.0, ...","[WATER, ALCOHOL DENAT., POTASSIUM CETYL PHOSPH...","[ANTIPLAQUE, SKIN CONDITIONING, SOLVENT, ANTIF...","[0.2007584439575184, 0.16060675516601475, 0.12...","[Ultra Facial Oil-Free Gel-Cream, White Lucent..."
1245,1468,Sun protect,KATE SOMERVILLE,Daily Deflector™ Waterlight Broad Spectrum SPF...,48,3.6,"[Water, Isododecane, Dimethicone, Butylocty...",0,0,0,0,0,"[92472, 34648, 33401, 31733, 74635, 75266, 340...","[WATER, ISODODECANE, DIMETHICONE, BUTYLOCTYL S...","[ANTIPLAQUE, SKIN CONDITIONING, SOLVENT, PERFU...","[0.2000265880940796, 0.1600212704752637, 0.128...",[Super Fluid UV Mineral Defense Titanium Dioxi...
1246,1469,Sun protect,VITA LIBERATA,Self Tan Dry Oil SPF 50,54,3.5,"[Water, Dihydroxyacetone, Glycerin, Scleroc...",0,0,0,0,0,"[92472, 75563, 34040, 58399, 35360, 55560, 848...","[WATER, DIHYDROXYACETONE, GLYCERIN, SCLEROCARY...","[ANTIPLAQUE, SKIN CONDITIONING, SOLVENT, HAIR ...","[0.20292444952078773, 0.16233955961663019, 0.1...","[Pro Light Self Tan Bronzing Mist, Self Tan Ex..."


In [86]:
df2.rename(columns = {"index": "Original Index"}).to_json("data/skincare_products_1.csv")

**NOTE:** NEED TO ADD BACK THE ROWS THAT WERE DROPPED